In this assignment, students will be using the K-nearest neighbors algorithm to predict
how many points NBA players scored in the 2013-2014 season.
A look at the data
Before we dive into the algorithm, let’s take a look at our data. Each row in the data
contains information on how a player performed in the 2013-2014 NBA season.
Download 'nba_2013.csv' file from this link:
https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0
Here are some selected columns from the data:
player - name of the player
pos - the position of the player
g - number of games the player was in
gs - number of games the player started
pts - total points the player scored
There are many more columns in the data, mostly containing information about average
player game performance over the course of the season. See this site for an explanation
of the rest of them.
We can read our dataset in and figure out which columns are present:

import pandas
with open("nba_2013.csv", 'r') as csvfile:
nba = pandas.read_csv(csvfile)

In [1]:
#Course: Data Science Masters - ML 7
#Title: Assignment 28.1            
#Tool: Jupyter notebook 
#Author: Krishna A
#Date: 14-07-2018 (DD/MM/YYYY)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()

'c:\\Temp\\acadgild'

In [6]:
Data = pd.read_csv('nba_2013.csv')

In [7]:
Data.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [8]:
Data.isnull().any()

player          False
pos             False
age             False
bref_team_id    False
g               False
gs              False
mp              False
fg              False
fga             False
fg.              True
x3p             False
x3pa            False
x3p.             True
x2p             False
x2pa            False
x2p.             True
efg.             True
ft              False
fta             False
ft.              True
orb             False
drb             False
trb             False
ast             False
stl             False
blk             False
tov             False
pf              False
pts             False
season          False
season_end      False
dtype: bool

In [9]:
Data.fillna(0, inplace=True)

In [10]:
Data.columns.tolist()

['player',
 'pos',
 'age',
 'bref_team_id',
 'g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fg.',
 'x3p',
 'x3pa',
 'x3p.',
 'x2p',
 'x2pa',
 'x2p.',
 'efg.',
 'ft',
 'fta',
 'ft.',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'season',
 'season_end']

In [11]:
#Dummification of the categorical values
Data_ = pd.get_dummies(data = Data, columns=['bref_team_id', 'pos','season'])
Data_.head()

,player,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,...,bref_team_id_UTA,bref_team_id_WAS,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG,season_2013-2014
0,Quincy Acy,23,63,0,847,66,141,0.468,4,15,...,0,0,0,0,0,0,0,1,0,1
1,Steven Adams,20,81,20,1197,93,185,0.503,0,0,...,0,0,1,0,0,0,0,0,0,1
2,Jeff Adrien,27,53,12,961,143,275,0.520,0,0,...,0,0,0,0,0,1,0,0,0,1
3,Arron Afflalo,28,73,73,2552,464,1011,0.459,128,300,...,0,0,0,0,0,0,0,0,1,1
4,Alexis Ajinca,25,56,30,951,136,249,0.546,0,1,...,0,0,1,0,0,0,0,0,0,1


In [12]:
Data_.sort_values(by =['pts'])

,player,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,...,bref_team_id_UTA,bref_team_id_WAS,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG,season_2013-2014
240,Darius Johnson-Odom,24,3,0,15,0,7,0.000,0,3,...,0,0,0,0,0,0,0,0,1,1
351,Dexter Pittman,25,2,0,3,0,1,0.000,0,0,...,0,0,1,0,0,0,0,0,0,1
90,Josh Childress,30,4,0,24,0,0,0.000,0,0,...,0,0,0,0,0,0,0,1,0,1
219,Royal Ivey,32,2,0,5,0,2,0.000,0,1,...,0,0,0,0,0,0,0,0,1,1
460,D.J. White,27,2,0,10,0,1,0.000,0,0,...,0,0,0,0,0,1,0,0,0,1
397,Chris Smith,26,2,0,2,0,0,0.000,0,0,...,0,0,0,0,0,0,1,0,0,1
461,Royce White,22,3,0,9,0,1,0.000,0,0,...,0,0,0,0,0,1,0,0,0,1
190,Elias Harris,24,2,0,11,0,1,0.000,0,0,...,0,0,0,0,0,0,0,1,0,1
210,Scotty Hopson,24,2,0,7,0,4,0.000,0,2,...,0,0,0,0,0,0,0,0,1,1
388,Mustafa Shakur,29,3,0,11,0,3,0.000,0,1,...,0,0,0,0,0,0,1,0,0,1


In [13]:
X = Data_.drop('pts', axis=1)
X = Data_.drop('player', axis=1)

In [14]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, Data_.pts)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [15]:
predict_test_Data = knn.predict(X)
print(accuracy_score(Data_.pts, predict_test_Data) *100, '%')

15.800415800415802 %


In [16]:
Data_.pts[:5]


0     171
1     265
2     362
3    1330
4     328
Name: pts, dtype: int64

In [17]:
predict_test_Data[:5]

array([ 132,  211,  328, 1245,  328], dtype=int64)

In [18]:
print(np.mean((Data_.pts - predict_test_Data)**2))

8768.787941787941


The above situation we r predicting & training on same dataset 
which leads to overfitting hence I am splitting the data into test
& train datasets.

In [19]:
X = Data_[[ 'age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'season_end', 'bref_team_id_ATL', 'bref_team_id_BOS', 'bref_team_id_BRK', 'bref_team_id_CHA', 'bref_team_id_CHI', 'bref_team_id_CLE', 'bref_team_id_DAL', 'bref_team_id_DEN', 'bref_team_id_DET', 'bref_team_id_GSW', 'bref_team_id_HOU', 'bref_team_id_IND', 'bref_team_id_LAC', 'bref_team_id_LAL', 'bref_team_id_MEM', 'bref_team_id_MIA', 'bref_team_id_MIL', 'bref_team_id_MIN', 'bref_team_id_NOP', 'bref_team_id_NYK', 'bref_team_id_OKC', 'bref_team_id_ORL', 'bref_team_id_PHI', 'bref_team_id_PHO', 'bref_team_id_POR', 'bref_team_id_SAC', 'bref_team_id_SAS', 'bref_team_id_TOR', 'bref_team_id_TOT', 'bref_team_id_UTA', 'bref_team_id_WAS', 'pos_C', 'pos_F', 'pos_G', 'pos_PF', 'pos_PG', 'pos_SF', 'pos_SG', 'season_2013-2014']]
Y = Data_['pts']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33,random_state =5)

In [20]:
knn = KNeighborsClassifier(n_neighbors=5)

In [21]:
knn.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [22]:

pred_KN_train = knn.predict(X_train)
pred_KN_test = knn.predict(X_test)

In [23]:
print(accuracy_score(Y_train, pred_KN_train) * 100, '%')
print(accuracy_score(Y_test, pred_KN_test) * 100, '%')

14.285714285714285 %
3.1446540880503147 %


In [24]:
pred_KN_test[:5]

array([454, 715, 303, 760, 286], dtype=int64)

In [25]:
Y_test[:5]

186    592
433    760
379    350
71     895
334    349
Name: pts, dtype: int64

In [26]:
np.mean((Y_test - pred_KN_test)**2)

11529.295597484277

#### Summary of analysis:
    1. Data from nba_2013.csv file.
    2. KNNeighbour model to fit the entire dataset and predicted the points that players 
        scored and calculated the accuracy score & mean squared error and predicted on the test dataset.
    3. Split the data into test & train datasets and used KNN classifier model 
        to fit the train & test datasets and make prediction on the test set and try to predict
        how many points players scored using 5 closest neighbours.
        